In [1]:
import os
import json
import pandas as pd
import numpy as np
import statistics
from helpers import *
import re
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv(r'Dataset_Big.csv')
df.head()

C:\Users\swak\AppData\Local\Temp\ipykernel_9404\52725572.py:1: DtypeWarning: Columns (228,302,307,309,316) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'Dataset_Big.csv')


,IMPORT_TABLE_size,strings_printabledist_31,strings_printabledist_88,.tls_vsize,subsystem,minor_image_version,RESOURCE_TABLE_virtual_address,.idata_vsize,.textbss_size,Dbgcore.dll,...,.idat_size,Libcrypto_num_funcs,mbedTLS,.txt_entropy,.UPX_props_len,CryptoAPI_num_funcs,.idat_props_len,.idat_vsize,.txt_size,.txt_vsize
0,200,858,965,8,WINDOWS_GUI,0,352256,2370,0,True,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,420,48,170,8,WINDOWS_GUI,20512,483328,8192,0,True,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,216,116,152,8,WINDOWS_GUI,0,90112,2242,0,True,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2370,613,653,8,WINDOWS_GUI,0,65536,2384,0,True,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,140,288,323,8,WINDOWS_GUI,0,49152,2384,0,True,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Save our feature list
feature_columns = list(df.columns)
feature_columns.pop(feature_columns.index('label'))
print(f"total number of Features: {len(feature_columns)}")
with open(os.path.join(os.getcwd(), 'assets', 'features.pkl'), 'wb') as f:
    pickle.dump(feature_columns, f)

total number of Features: 323


# Discard -1 rows

In [4]:
df = df[df['label'] != -1]

# Encode categorical columns

In [5]:
from sklearn.preprocessing import LabelEncoder


with open(os.path.join(os.getcwd(), 'assets', 'suspicious_imports.txt'), 'r') as f:
        sus_imports = f.readlines()
sus_imports = [re.sub(r'\n', '', i) for i in sus_imports]
boolean_columns = sus_imports + []
categorical_columns = ["subsystem", "magic", "machine"]

array_of_Label_Encoders = []
for col in categorical_columns:
    new_LE = LabelEncoder().fit(df[col])
    df[col] = new_LE.transform(df[col])
    array_of_Label_Encoders.append(new_LE)


for col in df.columns:
        if col in boolean_columns:
            df[col] = df[col].astype(bool)
            df[col].fillna(False)
            continue

with open(os.path.join(os.getcwd(), 'models', 'enc.pkl'), 'wb') as f:
    pickle.dump(array_of_Label_Encoders, f)



with open(os.path.join(os.getcwd(),'models','enc.pkl'), 'rb') as f:
        array_of_Label_Encoders = pickle.load(f)


# Random Forest

In [6]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

df_train_2 = df.copy()


feature_columns = list(df_train_2.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)


x_train, x_test, y_train, y_test = train_test_split(df_train_2[feature_columns], df_train_2['label'], test_size=0.3, shuffle=True)

rf_model = RandomForestClassifier(n_estimators=100, verbose=4).fit(x_train, y_train)

y_pred = rf_model.predict(x_test)
print(classification_report(y_test, y_pred))


rf_model.fit(x_test, y_test)

y_pred = rf_model.predict(x_test)
print(classification_report(y_test, y_pred))


with open(os.path.join(os.getcwd(), 'models', 'rf.pkl'), 'wb') as f:
    pickle.dump(rf_model, f)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.3s remaining:    0.0s


building tree 2 of 100


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   10.6s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.7s remaining:    0.0s


building tree 3 of 100


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    6.9s remaining:    0.0s


building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 10

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  3.8min finished


# SVM

In [18]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


df_train_1 = df.copy()

feature_columns = list(df_train_1.columns)
feature_columns.pop(feature_columns.index("label"))
if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)


x_train, x_test, y_train, y_test = train_test_split(df_train_1[feature_columns], df_train_1['label'], test_size=0.3, shuffle=True)

svm_model = SVC(kernel='poly', degree= 3, verbose=3).fit(x_train, y_train)

y_pred = svm_model.predict(x_test)
print(classification_report(y_test, y_pred))

svm_model = SVC(kernel='poly', degree= 3, verbose=3).fit(x_test, y_test)

with open(os.path.join(os.getcwd(), 'models', 'svm.pkl'), 'wb') as f:
    pickle.dump(svm_model, f)

[LibSVM]

# NN 1

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset



class MyNet(nn.Module):
    def __init__(self, num_features = 120):
        super(MyNet, self).__init__()
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, 512)
        self.batch_norm2 = nn.BatchNorm1d(512)
        self.dense2 = nn.Linear(512, 128)
        self.batch_norm3 = nn.BatchNorm1d(128)
        self.dense3 = nn.Linear(128, 8)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.batch_norm1(x.float())
        x = torch.tanh(self.dense1(x))
        x = self.batch_norm2(x.float())
        x = torch.tanh(self.dense2(x))
        x = self.batch_norm3(x.float())
        x = torch.tanh(self.dense3(x))
        x = self.softmax(x)
        return x

# Create an instance of the network
net = MyNet()

# Print the network architecture
print(net)




# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe_train, dataframe_labels):
        self.data = dataframe_train
        self.labels = dataframe_labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index, :]
        y = self.labels[index]
        return x, y



# Define the testing function
def test_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    device = next(model.parameters()).device  # Get the device of the model
    
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            
            # Forward pass
            outputs = model(inputs.to(device=device))
            # Get the predicted labels
            #preds = torch.round(torch.sigmoid(outputs))
            _, preds = torch.max(outputs, 1)  # Get the predicted labels
            
            # Collect the labels and predictions
            all_labels += list(labels.numpy().reshape((-1,1)))
            all_preds += list(preds.numpy().reshape((-1,1)))
    
    return np.asarray(all_labels), np.asarray(all_preds)



MyNet(
  (batch_norm1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=120, out_features=512, bias=True)
  (batch_norm2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense2): Linear(in_features=512, out_features=128, bias=True)
  (batch_norm3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense3): Linear(in_features=128, out_features=8, bias=True)
  (softmax): Softmax(dim=1)
)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"


df_train_3 = df.copy()


feature_columns = list(df_train_3.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)
    
num_features = len(feature_columns)


x_train, x_test, y_train, y_test = train_test_split(df_train_3.drop(columns=['label']), df_train_3['label'], test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)


# Split the data into features and labels
X = torch.from_numpy(np.asarray(x_train, dtype=np.int64))
y = torch.from_numpy(np.asarray(y_train, dtype=bool)).type(torch.int64)

# Split the data into features and labels
X_val = torch.from_numpy(np.asarray(x_val, dtype=np.int64))
y_val = torch.from_numpy(np.asarray(y_val, dtype=bool)).type(torch.int64)

# Create instances of the dataset and data loader
dataset = MyDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)



# Create an instance of the network
net = MyNet(num_features).to(device=device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device=device)
optimizer = optim.Adam(net.parameters(), lr=0.0001)#, weight_decay=0.0001)

# Training loop
for epoch in range(70):  # Replace 10 with the desired number of epochs
    running_loss = 0.0
    running_corrects = 0  # Counter for correct predictions
    
    for inputs, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs.to(device=device))
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate the predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        corrects = torch.sum(preds == labels)  # Count the number of correct predictions
        running_corrects += corrects.item()
    
    # Calculate the running accuracy on the training set
    train_accuracy = running_corrects / len(dataset)
     
    # Calculate the validation accuracy
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()
            val_total += labels.size(0)
    val_accuracy = val_corrects / val_total
    
    # Print the average loss and accuracy for the epoch
    print(f'Epoch {epoch+1}: Loss = {running_loss/len(dataloader):.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}')

# Save the trained model
#torch.save(net.state_dict(), 'trained_model.pt')

with open(os.path.join(os.getcwd(), 'models', 'NN1.pkl'), 'wb') as f:
     pickle.dump(net.cpu(), f)




# Split the data into features and labels
X_test = torch.from_numpy(np.asarray(x_test, dtype=np.int64))
Y_test = torch.from_numpy(np.asarray(y_test, dtype=bool)).type(torch.int64)


test_dataset = MyDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Test the model
true_labels, predicted_labels = test_model(net.to(device=device), test_dataloader)

# Import the necessary libraries for classification report
from sklearn.metrics import classification_report

# Print the classification report
target_names = ['class_0', 'class_1']  # Replace with appropriate class names
print(classification_report(true_labels, predicted_labels, labels=np.unique(true_labels)))


Epoch 1: Loss = 1.9132, Train Accuracy = 0.6080, Val Accuracy = 0.6587
Epoch 2: Loss = 1.8472, Train Accuracy = 0.6919, Val Accuracy = 0.7053
Epoch 3: Loss = 1.8261, Train Accuracy = 0.7149, Val Accuracy = 0.7291
Epoch 4: Loss = 1.8144, Train Accuracy = 0.7318, Val Accuracy = 0.7411
Epoch 5: Loss = 1.8083, Train Accuracy = 0.7431, Val Accuracy = 0.7658
Epoch 6: Loss = 1.7969, Train Accuracy = 0.7646, Val Accuracy = 0.7733
Epoch 7: Loss = 1.7935, Train Accuracy = 0.7716, Val Accuracy = 0.7801
Epoch 8: Loss = 1.7909, Train Accuracy = 0.7788, Val Accuracy = 0.7848
Epoch 9: Loss = 1.7884, Train Accuracy = 0.7855, Val Accuracy = 0.7880
Epoch 10: Loss = 1.7861, Train Accuracy = 0.7915, Val Accuracy = 0.8018
Epoch 11: Loss = 1.7843, Train Accuracy = 0.8000, Val Accuracy = 0.8047
Epoch 12: Loss = 1.7801, Train Accuracy = 0.8168, Val Accuracy = 0.8169
Epoch 13: Loss = 1.7777, Train Accuracy = 0.8224, Val Accuracy = 0.8216
Epoch 14: Loss = 1.7757, Train Accuracy = 0.8266, Val Accuracy = 0.8282
E

## NN2
### same as base NN model, but without any batch normalization

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset



class MyNet(nn.Module):
    def __init__(self, num_features = 120):
        super(MyNet, self).__init__()

        self.dense1 = nn.Linear(num_features, 512)
        self.dense2 = nn.Linear(512, 128)
        self.dense3 = nn.Linear(128, 8)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = torch.tanh(self.dense1(x))
        x = torch.tanh(self.dense2(x))
        x = torch.tanh(self.dense3(x))
        x = self.softmax(x)
        return x

# Create an instance of the network
net = MyNet()

# Print the network architecture
print(net)

# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe_train, dataframe_labels):
        self.data = dataframe_train
        self.labels = dataframe_labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index, :]
        y = self.labels[index]
        return x, y

MyNet(
  (dense1): Linear(in_features=120, out_features=512, bias=True)
  (dense2): Linear(in_features=512, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=8, bias=True)
  (softmax): Softmax(dim=1)
)


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"


df_train_3 = df.copy()


feature_columns = list(df_train_3.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)
    
num_features = len(feature_columns)


x_train, x_test, y_train, y_test = train_test_split(df_train_3.drop(columns=['label']), df_train_3['label'], test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)


# Split the data into features and labels
X = torch.from_numpy(np.asarray(x_train, dtype=np.int64)).type(torch.float)
y = torch.from_numpy(np.asarray(y_train, dtype=bool)).type(torch.int64)

# Split the data into features and labels
X_val = torch.from_numpy(np.asarray(x_val, dtype=np.int64)).type(torch.float)
y_val = torch.from_numpy(np.asarray(y_val, dtype=bool)).type(torch.int64)

# Create instances of the dataset and data loader
dataset = MyDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)



# Create an instance of the network
net = MyNet(num_features).to(device=device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device=device)
optimizer = optim.Adam(net.parameters(), lr=0.0001)#, weight_decay=0.0001)

# Training loop
for epoch in range(70):  # Replace 10 with the desired number of epochs
    running_loss = 0.0
    running_corrects = 0  # Counter for correct predictions
    
    for inputs, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs.to(device=device))
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate the predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        corrects = torch.sum(preds == labels)  # Count the number of correct predictions
        running_corrects += corrects.item()
    
    # Calculate the running accuracy on the training set
    train_accuracy = running_corrects / len(dataset)
     
    # Calculate the validation accuracy
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()
            val_total += labels.size(0)
    val_accuracy = val_corrects / val_total
    
    # Print the average loss and accuracy for the epoch
    print(f'Epoch {epoch+1}: Loss = {running_loss/len(dataloader):.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}')


with open(os.path.join(os.getcwd(), 'models', 'NN2.pkl'), 'wb') as f:
     pickle.dump(net.cpu(), f)




# Split the data into features and labels
X_test = torch.from_numpy(np.asarray(x_test, dtype=np.int64)).type(torch.float)
Y_test = torch.from_numpy(np.asarray(y_test, dtype=bool)).type(torch.int64)


test_dataset = MyDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Test the model
true_labels, predicted_labels = test_model(net.to(device=device), test_dataloader)

# Import the necessary libraries for classification report
from sklearn.metrics import classification_report

# Print the classification report
target_names = ['class_0', 'class_1']  # Replace with appropriate class names
print(classification_report(true_labels, predicted_labels, labels=np.unique(true_labels)))


Epoch 1: Loss = 1.8663, Train Accuracy = 0.5118, Val Accuracy = 0.5036
Epoch 2: Loss = 1.8545, Train Accuracy = 0.5008, Val Accuracy = 0.5037
Epoch 3: Loss = 1.8537, Train Accuracy = 0.5008, Val Accuracy = 0.5037
Epoch 4: Loss = 1.8534, Train Accuracy = 0.5009, Val Accuracy = 0.5038
Epoch 5: Loss = 1.8533, Train Accuracy = 0.5010, Val Accuracy = 0.5039
Epoch 6: Loss = 1.8532, Train Accuracy = 0.5280, Val Accuracy = 0.5337
Epoch 7: Loss = 1.8532, Train Accuracy = 0.5356, Val Accuracy = 0.5332
Epoch 8: Loss = 1.8531, Train Accuracy = 0.5354, Val Accuracy = 0.5332
Epoch 9: Loss = 1.8531, Train Accuracy = 0.5355, Val Accuracy = 0.5333
Epoch 10: Loss = 1.8530, Train Accuracy = 0.5355, Val Accuracy = 0.5333
Epoch 11: Loss = 1.8530, Train Accuracy = 0.5356, Val Accuracy = 0.5333
Epoch 12: Loss = 1.8530, Train Accuracy = 0.5356, Val Accuracy = 0.5334
Epoch 13: Loss = 1.8530, Train Accuracy = 0.5356, Val Accuracy = 0.5334
Epoch 14: Loss = 1.8530, Train Accuracy = 0.5356, Val Accuracy = 0.5333
E

# NN3
### same as base model, but with Leaky Relu instead of the tanh activation functions

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset



class MyNet(nn.Module):
    def __init__(self, num_features = 120):
        super(MyNet, self).__init__()
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, 512)
        self.lrelu1 = nn.LeakyReLU()
        self.batch_norm2 = nn.BatchNorm1d(512)
        self.dense2 = nn.Linear(512, 128)
        self.lrelu2 = nn.LeakyReLU()
        self.batch_norm3 = nn.BatchNorm1d(128)
        self.dense3 = nn.Linear(128, 8)
        self.lrelu3 = nn.LeakyReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.batch_norm1(x.float())
        x = self.lrelu1(self.dense1(x))
        x = self.batch_norm2(x.float())
        x = self.lrelu2(self.dense2(x))
        x = self.batch_norm3(x.float())
        x = self.lrelu3(self.dense3(x))
        x = self.softmax(x)
        return x

# Create an instance of the network
net = MyNet()

# Print the network architecture
print(net)




# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe_train, dataframe_labels):
        self.data = dataframe_train
        self.labels = dataframe_labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index, :]
        y = self.labels[index]
        return x, y

MyNet(
  (batch_norm1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=120, out_features=512, bias=True)
  (lrelu1): LeakyReLU(negative_slope=0.01)
  (batch_norm2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense2): Linear(in_features=512, out_features=128, bias=True)
  (lrelu2): LeakyReLU(negative_slope=0.01)
  (batch_norm3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense3): Linear(in_features=128, out_features=8, bias=True)
  (lrelu3): LeakyReLU(negative_slope=0.01)
  (softmax): Softmax(dim=1)
)


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"


df_train_3 = df.copy()


feature_columns = list(df_train_3.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)
    
num_features = len(feature_columns)


x_train, x_test, y_train, y_test = train_test_split(df_train_3.drop(columns=['label']), df_train_3['label'], test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)


# Split the data into features and labels
X = torch.from_numpy(np.asarray(x_train, dtype=np.int64))
y = torch.from_numpy(np.asarray(y_train, dtype=bool)).type(torch.int64)

# Split the data into features and labels
X_val = torch.from_numpy(np.asarray(x_val, dtype=np.int64))
y_val = torch.from_numpy(np.asarray(y_val, dtype=bool)).type(torch.int64)

# Create instances of the dataset and data loader
dataset = MyDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)



# Create an instance of the network
net = MyNet(num_features).to(device=device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device=device)
optimizer = optim.Adam(net.parameters(), lr=0.0001)#, weight_decay=0.0001)

# Training loop
for epoch in range(70):  # Replace 10 with the desired number of epochs
    running_loss = 0.0
    running_corrects = 0  # Counter for correct predictions
    
    for inputs, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs.to(device=device))
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate the predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        corrects = torch.sum(preds == labels)  # Count the number of correct predictions
        running_corrects += corrects.item()
    
    # Calculate the running accuracy on the training set
    train_accuracy = running_corrects / len(dataset)
     
    # Calculate the validation accuracy
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()
            val_total += labels.size(0)
    val_accuracy = val_corrects / val_total
    
    # Print the average loss and accuracy for the epoch
    print(f'Epoch {epoch+1}: Loss = {running_loss/len(dataloader):.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}')

# Save the trained model
#torch.save(net.state_dict(), 'trained_model.pt')

with open(os.path.join(os.getcwd(), 'models', 'NN3.pkl'), 'wb') as f:
     pickle.dump(net.cpu(), f)




# Split the data into features and labels
X_test = torch.from_numpy(np.asarray(x_test, dtype=np.int64)).type(torch.float)
Y_test = torch.from_numpy(np.asarray(y_test, dtype=bool)).type(torch.int64)


test_dataset = MyDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Test the model
true_labels, predicted_labels = test_model(net.to(device=device), test_dataloader)

# Import the necessary libraries for classification report
from sklearn.metrics import classification_report

# Print the classification report
target_names = ['class_0', 'class_1']  # Replace with appropriate class names
print(classification_report(true_labels, predicted_labels, labels=np.unique(true_labels)))

Epoch 1: Loss = 1.6602, Train Accuracy = 0.6768, Val Accuracy = 0.7649
Epoch 2: Loss = 1.5088, Train Accuracy = 0.7807, Val Accuracy = 0.7947
Epoch 3: Loss = 1.4764, Train Accuracy = 0.8043, Val Accuracy = 0.8059
Epoch 4: Loss = 1.4607, Train Accuracy = 0.8155, Val Accuracy = 0.8147
Epoch 5: Loss = 1.4512, Train Accuracy = 0.8227, Val Accuracy = 0.8183
Epoch 6: Loss = 1.4452, Train Accuracy = 0.8275, Val Accuracy = 0.8230
Epoch 7: Loss = 1.4406, Train Accuracy = 0.8314, Val Accuracy = 0.8251
Epoch 8: Loss = 1.4365, Train Accuracy = 0.8349, Val Accuracy = 0.8290
Epoch 9: Loss = 1.4293, Train Accuracy = 0.8433, Val Accuracy = 0.8506
Epoch 10: Loss = 1.4135, Train Accuracy = 0.8616, Val Accuracy = 0.8542
Epoch 11: Loss = 1.4105, Train Accuracy = 0.8637, Val Accuracy = 0.8558
Epoch 12: Loss = 1.4088, Train Accuracy = 0.8657, Val Accuracy = 0.8573
Epoch 13: Loss = 1.4062, Train Accuracy = 0.8683, Val Accuracy = 0.8599
Epoch 14: Loss = 1.4039, Train Accuracy = 0.8705, Val Accuracy = 0.8609
E

# NN 4
### same base model but with an extra dense layer and weight decay = 0.000001

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset



class MyNet(nn.Module):
    def __init__(self, num_features = 120):
        super(MyNet, self).__init__()
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, 512)
        self.batch_norm2 = nn.BatchNorm1d(512)
        self.dense2 = nn.Linear(512, 512)
        self.batch_norm3 = nn.BatchNorm1d(512)
        self.dense3 = nn.Linear(512, 128)
        self.batch_norm4 = nn.BatchNorm1d(128)
        self.dense4 = nn.Linear(128, 8)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.batch_norm1(x.float())
        x = torch.tanh(self.dense1(x))
        x = self.batch_norm2(x.float())
        x = torch.tanh(self.dense2(x))
        x = self.batch_norm3(x.float())
        x = torch.tanh(self.dense3(x))
        x = self.batch_norm4(x.float())
        x = torch.tanh(self.dense4(x))
        x = self.softmax(x)
        return x

# Create an instance of the network
net = MyNet()

# Print the network architecture
print(net)




# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe_train, dataframe_labels):
        self.data = dataframe_train
        self.labels = dataframe_labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index, :]
        y = self.labels[index]
        return x, y

MyNet(
  (batch_norm1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=120, out_features=512, bias=True)
  (batch_norm2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense2): Linear(in_features=512, out_features=512, bias=True)
  (batch_norm3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense3): Linear(in_features=512, out_features=128, bias=True)
  (batch_norm4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense4): Linear(in_features=128, out_features=8, bias=True)
  (softmax): Softmax(dim=1)
)


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"


df_train_3 = df.copy()


feature_columns = list(df_train_3.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)
    
num_features = len(feature_columns)


x_train, x_test, y_train, y_test = train_test_split(df_train_3.drop(columns=['label']), df_train_3['label'], test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)


# Split the data into features and labels
X = torch.from_numpy(np.asarray(x_train, dtype=np.int64))
y = torch.from_numpy(np.asarray(y_train, dtype=bool)).type(torch.int64)

# Split the data into features and labels
X_val = torch.from_numpy(np.asarray(x_val, dtype=np.int64))
y_val = torch.from_numpy(np.asarray(y_val, dtype=bool)).type(torch.int64)

# Create instances of the dataset and data loader
dataset = MyDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)



# Create an instance of the network
net = MyNet(num_features).to(device=device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device=device)
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.000001)

# Training loop
for epoch in range(70):  # Replace 10 with the desired number of epochs
    running_loss = 0.0
    running_corrects = 0  # Counter for correct predictions
    
    for inputs, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs.to(device=device))
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate the predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        corrects = torch.sum(preds == labels)  # Count the number of correct predictions
        running_corrects += corrects.item()
    
    # Calculate the running accuracy on the training set
    train_accuracy = running_corrects / len(dataset)
     
    # Calculate the validation accuracy
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()
            val_total += labels.size(0)
    val_accuracy = val_corrects / val_total
    
    # Print the average loss and accuracy for the epoch
    print(f'Epoch {epoch+1}: Loss = {running_loss/len(dataloader):.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}')

# Save the trained model
#torch.save(net.state_dict(), 'trained_model.pt')

with open(os.path.join(os.getcwd(), 'models', 'NN4.pkl'), 'wb') as f:
     pickle.dump(net.cpu(), f)




# Split the data into features and labels
X_test = torch.from_numpy(np.asarray(x_test, dtype=np.int64)).type(torch.float)
Y_test = torch.from_numpy(np.asarray(y_test, dtype=bool)).type(torch.int64)


test_dataset = MyDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Test the model
true_labels, predicted_labels = test_model(net.to(device=device), test_dataloader)

# Import the necessary libraries for classification report
from sklearn.metrics import classification_report

# Print the classification report
target_names = ['class_0', 'class_1']  # Replace with appropriate class names
print(classification_report(true_labels, predicted_labels, labels=np.unique(true_labels)))


Epoch 1: Loss = 1.8330, Train Accuracy = 0.7044, Val Accuracy = 0.7471
Epoch 2: Loss = 1.7979, Train Accuracy = 0.7688, Val Accuracy = 0.7876
Epoch 3: Loss = 1.7748, Train Accuracy = 0.8228, Val Accuracy = 0.8368
Epoch 4: Loss = 1.7629, Train Accuracy = 0.8575, Val Accuracy = 0.8610
Epoch 5: Loss = 1.7569, Train Accuracy = 0.8720, Val Accuracy = 0.8718
Epoch 6: Loss = 1.7525, Train Accuracy = 0.8826, Val Accuracy = 0.8816
Epoch 7: Loss = 1.7485, Train Accuracy = 0.8915, Val Accuracy = 0.8854
Epoch 8: Loss = 1.7460, Train Accuracy = 0.8975, Val Accuracy = 0.8955
Epoch 9: Loss = 1.7442, Train Accuracy = 0.9021, Val Accuracy = 0.8980
Epoch 10: Loss = 1.7424, Train Accuracy = 0.9062, Val Accuracy = 0.9013
Epoch 11: Loss = 1.7413, Train Accuracy = 0.9088, Val Accuracy = 0.9013
Epoch 12: Loss = 1.7397, Train Accuracy = 0.9129, Val Accuracy = 0.9074
Epoch 13: Loss = 1.7387, Train Accuracy = 0.9154, Val Accuracy = 0.9079
Epoch 14: Loss = 1.7373, Train Accuracy = 0.9193, Val Accuracy = 0.9102
E

# NN 5
### same as NN 4 but with bigger learning rate, and with weight decay

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"


df_train_3 = df.copy()


feature_columns = list(df_train_3.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)
    
num_features = len(feature_columns)


x_train, x_test, y_train, y_test = train_test_split(df_train_3.drop(columns=['label']), df_train_3['label'], test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)


# Split the data into features and labels
X = torch.from_numpy(np.asarray(x_train, dtype=np.int64))
y = torch.from_numpy(np.asarray(y_train, dtype=bool)).type(torch.int64)

# Split the data into features and labels
X_val = torch.from_numpy(np.asarray(x_val, dtype=np.int64))
y_val = torch.from_numpy(np.asarray(y_val, dtype=bool)).type(torch.int64)

# Create instances of the dataset and data loader
dataset = MyDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)



# Create an instance of the network
net = MyNet(num_features).to(device=device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device=device)
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.000001)

# Training loop
for epoch in range(70):  # Replace 10 with the desired number of epochs
    running_loss = 0.0
    running_corrects = 0  # Counter for correct predictions
    
    for inputs, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs.to(device=device))
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate the predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        corrects = torch.sum(preds == labels)  # Count the number of correct predictions
        running_corrects += corrects.item()
    
    # Calculate the running accuracy on the training set
    train_accuracy = running_corrects / len(dataset)
     
    # Calculate the validation accuracy
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()
            val_total += labels.size(0)
    val_accuracy = val_corrects / val_total
    
    # Print the average loss and accuracy for the epoch
    print(f'Epoch {epoch+1}: Loss = {running_loss/len(dataloader):.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}')

# Save the trained model
#torch.save(net.state_dict(), 'trained_model.pt')

with open(os.path.join(os.getcwd(), 'models', 'NN5.pkl'), 'wb') as f:
     pickle.dump(net.cpu(), f)




# Split the data into features and labels
X_test = torch.from_numpy(np.asarray(x_test, dtype=np.int64)).type(torch.float)
Y_test = torch.from_numpy(np.asarray(y_test, dtype=bool)).type(torch.int64)


test_dataset = MyDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Test the model
true_labels, predicted_labels = test_model(net.to(device=device), test_dataloader)

# Import the necessary libraries for classification report
from sklearn.metrics import classification_report

# Print the classification report
target_names = ['class_0', 'class_1']  # Replace with appropriate class names
print(classification_report(true_labels, predicted_labels, labels=np.unique(true_labels)))


Epoch 1: Loss = 1.8401, Train Accuracy = 0.6794, Val Accuracy = 0.7226
Epoch 2: Loss = 1.7965, Train Accuracy = 0.7662, Val Accuracy = 0.7916
Epoch 3: Loss = 1.7766, Train Accuracy = 0.8184, Val Accuracy = 0.8360
Epoch 4: Loss = 1.7633, Train Accuracy = 0.8564, Val Accuracy = 0.8625
Epoch 5: Loss = 1.7563, Train Accuracy = 0.8748, Val Accuracy = 0.8749
Epoch 6: Loss = 1.7523, Train Accuracy = 0.8844, Val Accuracy = 0.8814
Epoch 7: Loss = 1.7489, Train Accuracy = 0.8924, Val Accuracy = 0.8905
Epoch 8: Loss = 1.7460, Train Accuracy = 0.8998, Val Accuracy = 0.8955
Epoch 9: Loss = 1.7445, Train Accuracy = 0.9032, Val Accuracy = 0.8968
Epoch 10: Loss = 1.7428, Train Accuracy = 0.9066, Val Accuracy = 0.9001
Epoch 11: Loss = 1.7406, Train Accuracy = 0.9110, Val Accuracy = 0.9046
Epoch 12: Loss = 1.7390, Train Accuracy = 0.9146, Val Accuracy = 0.9065
Epoch 13: Loss = 1.7378, Train Accuracy = 0.9181, Val Accuracy = 0.9100
Epoch 14: Loss = 1.7371, Train Accuracy = 0.9198, Val Accuracy = 0.9100
E

# GBT

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

df_train_2 = df.copy()


feature_columns = list(df_train_2.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)


x_train, x_test, y_train, y_test = train_test_split(df_train_2[feature_columns], df_train_2['label'], test_size=0.3, shuffle=True)

gbt_model = GradientBoostingClassifier(init=AdaBoostClassifier(), learning_rate=0.001, n_estimators= 300, verbose=4).fit(x_train, y_train)

y_pred = gbt_model.predict(x_test)
print(classification_report(y_test, y_pred))

gbt_model.fit(x_test, y_test)

with open(os.path.join(os.getcwd(), 'models', 'gbt.pkl'), 'wb') as f:
    pickle.dump(gbt_model, f)

      Iter       Train Loss   Remaining Time 
         1           1.3424           85.44m
         2           1.3418           85.66m
         3           1.3411           84.88m
         4           1.3405           83.99m
         5           1.3399           83.88m
         6           1.3393           83.56m
         7           1.3386           82.23m
         8           1.3380           81.24m
         9           1.3374           80.36m
        10           1.3368           79.67m
        11           1.3362           79.12m
        12           1.3355           78.43m
        13           1.3349           77.83m
        14           1.3343           77.34m
        15           1.3337           77.19m
        16           1.3331           77.03m
        17           1.3325           76.94m
        18           1.3319           76.78m
        19           1.3312           76.69m
        20           1.3306           76.32m
        21           1.3300           76.22m
        2

# light GBM

In [17]:
import lightgbm as lgb


df_train_2 = df.copy()


feature_columns = list(df_train_2.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)


x_train, x_test, y_train, y_test = train_test_split(df_train_2[feature_columns], df_train_2['label'], test_size=0.3, shuffle=True)

lgb_model = lgb.LGBMClassifier(verbose=4, n_estimators=100).fit(x_train, y_train)

y_pred = lgb_model.predict(x_test)
print(classification_report(y_test, y_pred))

lgb.fit(x_test, y_test)

with open(os.path.join(os.getcwd(), 'models', 'lgb.pkl'), 'wb') as f:
    pickle.dump(lgb_model, f)

[LightGBM] [Info] Number of positive: 209901, number of negative: 210099
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.916580
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.489503
[LightGBM] [Debug] init for col-wise cost 0.123866 seconds, init for row-wise cost 0.490855 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.561478 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46845
[LightGBM] [Info] Number of data points in the train set: 420000, number of used features: 312
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499764 -> initscore=-0.000943
[LightGBM] [Info] Start training from score -0.000943
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 